In [ ]:
#!/usr/bin/env python
# coding: utf-8

"""
Descriptor Convolutional Neural Network
"""

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Install packages

In [ ]:
! pip install nvgpu
! pip install py-cpuinfo
! pip install timer
! python -m pip install psutil==5.7.2 --user

# Check if is running on Colab

In [ ]:
use_colab = 'google.colab' in str(get_ipython())

if use_colab:
    # Define base path
    BASE_PATH = '/content/gdrive/My Drive/Colab Notebooks/DESC-NET/src/'

    # Import Libraries from colab
    from google.colab import drive
    from google.colab import output
    import IPython
    
    # Mount Google Drive for fast, responsible access to files
    drive.mount('/content/gdrive')
    
    message_use_colab = '[INFO] Running on Colab:\n'
else:
    message_use_colab = '[INFO] Not running on Colab:\n'
    pass

# Colab Runtime to prevent from disconnecting

In [ ]:
if use_colab:
    display(IPython.display.Javascript('''
    function ClickConnect(){
        btn = document.querySelector("colab-connect-button")
        if (btn != null){
          console.log("Click colab-connect-button"); 
          btn.click() 
        }
        
        btn = document.getElementById('ok')
        if (btn != null){
          console.log("Click reconnect"); 
          btn.click() 
        }
      }
        
    setInterval(ClickConnect, 60000)'''))
else:
    pass

# Import Libraries

In [ ]:
from PIL import Image
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2 as cv
import glob
import json
import numpy as np
import os
import pickle
import random
import tensorflow as tf
import time
import timeit
import zipfile

# Import Learning Rate Schedulers

In [ ]:
if use_colab:
    SCHEDULERS = 'lr_schedulers.zip'

    SCHEDULERS_PATH =  "/".join([BASE_PATH, SCHEDULERS])

    zip_object = zipfile.ZipFile(file = SCHEDULERS_PATH,
                                mode = 'r')

    zip_object.extractall()

    zip_object.close

from lr_schedulers.clr_callback import CyclicLR
from lr_schedulers.learning_rate_schedulers import StepDecay

# Import Custom Libraries

# ImportError: cannot import name 'bytes2human' from 'psutil._common' (/usr/local/lib/python3.7/dist-packages/psutil/_common.py)

Just go to Runtime >>> Reset all runtimes...

In [ ]:
# To import and use the custom imports, the folder need
# to be in se same directory to de file (in colab) 
if use_colab:
    # Unzip in Colab Workspace
    UTILITIES = 'utilities.zip'

    UTILITIES_PATH =  "/".join([BASE_PATH, UTILITIES])

    zip_object = zipfile.ZipFile(file = UTILITIES_PATH,
                                mode = 'r')

    zip_object.extractall()

    zip_object.close

from utilities.dataset import(#manipulate_pandas,
                              create_dataset
)

from utilities.descriptors import find_detector_or_descriptor

from utilities import global_variables

from utilities.logfile import(close_logfile,
                              open_logfile,
                              print_descriptor_initialized_info,
                              print_detector_initialized_info,
                              print_system_info,
                              print_packages_info
)

from utilities.timer import Timer

from utilities.visualizations import(#save_dataset_distribution,
                                     save_architecture_visualization,
                                     save_model_accuracy_history,
                                     save_model_loss_history,
                                     save_confusion_matrix,
)


from utilities.local_feature_detection import local_feature_detection

from utilities.local_descriptor_convolution import * # TODO: Arrumar e importar todos... depois...

from utilities.residual_neural_network import ResNet

from utilities.model_functions import(training,
                                      evaluate,
                                      predict)

# Import config.json
## Used to load the user configurations

In [ ]:
if use_colab:
    # Unzip in Colab Workspace
    JSON = 'config.zip'

    JSON_PATH =  "/".join([BASE_PATH, JSON])

    zip_object = zipfile.ZipFile(file = JSON_PATH,
                                 mode = 'r')

    zip_object.extractall()

    zip_object.close 

JSON_PATH = 'config.json'

# Load the user configurations
with open(JSON_PATH, encoding='utf8') as f:    
    config = json.load(f)

# Config variables

In [ ]:
# Stores the dataset's name
DATASET = config['dataset']

try:
    assert(DATASET == 'MNIST' or
           DATASET == 'JAFFE' or
           DATASET == 'Extended-CK+' or
           DATASET == 'CIFAR-10' or
           DATASET == 'CIFAR-100' or
           DATASET == 'FEI' or
           DATASET == 'FER-2013' or
           DATASET == 'ILSVRC-2017')
except AssertionError as e:
    raise(AssertionError('`DATASET` cannot be different from MNIST or JAFFE or Extended-CK+ or CIFAR-10 or CIFAR-100 or FEI or FER-2013 or ILSVRC-2017'))

# Path of training-set
DATA_TRAIN_IMAGES = "/".join([config['dataset'], config['data_train_images']])

# Path of test-set
DATA_TEST_IMAGES = "/".join([config['dataset'], config['data_test_images']])

# Stores the detector's name
DETECTOR = config['detector']

try:
    assert(DETECTOR == 'SIFT' or
           DETECTOR == 'SURF' or
           DETECTOR == 'KAZE' or
           DETECTOR == 'ORB' or
           DETECTOR == 'BRISK' or
           DETECTOR == 'AKAZE')
except AssertionError as e:
    raise(AssertionError('`DETECTOR` cannot be different from SIFT or SURF or KAZE or ORB or BRISK or AKAZE'))

# Stores the descriptor's name
DESCRIPTOR = config['descriptor']

try:
    assert(DESCRIPTOR == 'SIFT' or
           DESCRIPTOR == 'SURF' or
           DESCRIPTOR == 'KAZE' or
           DESCRIPTOR == 'BRIEF' or
           DESCRIPTOR == 'ORB' or
           DESCRIPTOR == 'BRISK' or
           DESCRIPTOR == 'AKAZE' or
           DESCRIPTOR == 'FREAK')
except AssertionError as e:
    raise(AssertionError('`DESCRIPTOR` cannot be different from SIFT or SURF or KAZE or BRIEF or ORB or BRISK or AKAZE or FREAK'))

# Stores the reducer's name
#REDUCER = config['reducer']

# Stores the architecture's name of CNN
ARCHITECTURE = config['architecture']

try:
    assert(ARCHITECTURE == 'DescNet-50' or
           ARCHITECTURE == 'DescNet-101' or
           ARCHITECTURE == 'DescNet-152' or
           ARCHITECTURE == 'ResNet-50' or
           ARCHITECTURE == 'ResNet-101' or
           ARCHITECTURE == 'ResNet-152')
except AssertionError as e:
    raise(AssertionError('`ARCHITECTURE` cannot be different from DescNet-50 or DescNet-101 or DescNet-152 or ResNet-50 or ResNet-101 or ResNet-152'))

# Store's desc block if used
USE_DESC_BLOCK = config['use_desc_block']
 
# Define base path
if not use_colab:
    BASE_PATH = config['main_path']
    
# Output path
OUTPUT = "/".join(['Outputs', config['dataset'], config['architecture'], config['descriptor']])

# Initializing global variables

In [ ]:
global_variables.initialize()

# Simple line to difide dome steps of this project
global_variables.LINE

# Absolute output path
global_variables.OUTPUT_PATH = "/".join([BASE_PATH, OUTPUT])

# Create Output path if not exists

In [ ]:
try:
    os.makedirs(name = global_variables.OUTPUT_PATH)
    print('\n[INFO]: `global_variables.OUTPUT_PATH` does not exist... creating...')
except FileExistsError:
    # global_variables.OUTPUT_PATH already exists
    pass

# Open logfile to start write

In [ ]:
open_logfile()

# Print if is running or not on Colab

In [ ]:
print(message_use_colab, file = global_variables.LOGFILE)

# Print current CPU, GPU, and RAM

In [ ]:
print_system_info()

# Set current Distribution Strategy

In [ ]:
if use_colab:
    # Try to run on TPU
    # Detect hardware, return appropriate distribution strategy
    try:
        # Get a handle to the attached TPU
        TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection

        # print('Running on TPU', tpu.cluster_spec().as_dict()['worker'])
    except ValueError:
        TPU = None

    if TPU:
        # Connect to the TPU handle 
        tf.config.experimental_connect_to_cluster(TPU)

        # And initialise it
        tf.tpu.experimental.initialize_tpu_system(TPU)

        # Set the distribution strategy
        strategy = tf.distribute.experimental.TPUStrategy(TPU)
        
        print(f'[INFO] TPU Strategy adopted:\n', file = global_variables.LOGFILE)
        
        print(f'[INFO] Number of devices: {strategy.num_replicas_in_sync}\n', file = global_variables.LOGFILE)
        
        print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)
        
elif tf.config.list_physical_devices('GPU'):
    strategy = tf.distribute.MirroredStrategy()
    
    print(f'[INFO] GPU Strategy adopted:\n', file = global_variables.LOGFILE)
    
    print(f'[INFO] Number of devices: {strategy.num_replicas_in_sync}\n', file = global_variables.LOGFILE)
    
    print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)

# Use default strategy
else:
    strategy = tf.distribute.get_strategy()
    
    print(f'[INFO] Default Strategy adopted:\n', file = global_variables.LOGFILE)
    
    print(f'[INFO] Number of devices: {strategy.num_replicas_in_sync}\n', file = global_variables.LOGFILE)
    
    print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)

# Print Python, OpenCV, TensorFlow, and Keras versions

In [ ]:
print_packages_info()

# Unzip outside Dataset

In [ ]:
DATASET_PATH = '/'.join([BASE_PATH, 'Datasets', DATASET + '.zip'])

zip_object = zipfile.ZipFile(file = DATASET_PATH,
                             mode = 'r')

zip_object.extractall('./')

zip_object.close

# Dimensions of the data

In [ ]:
# Classes info
CLASSES = os.listdir(DATA_TRAIN_IMAGES)

for i in range(1):
    TEST_FOLDER = '/'.join([DATA_TRAIN_IMAGES, CLASSES[0]])
    
    image_name = random.choice(os.listdir(TEST_FOLDER))

    image_path = "/".join([TEST_FOLDER, image_name])

    image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

    # Images size
    IMAGE_HEIGHT, IMAGE_WIDTH = image.shape

# Images are stored in one-dimensional arrays of this length
IMAGE_SIZE_FLAT = IMAGE_HEIGHT * IMAGE_WIDTH

# Tuple with height and width of images used to reshape arrays
IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH)

# Number of classes
N_CLASSES = len(CLASSES)

# Number of colour channels for the images
# Channels mean number of primary colors   
if image.ndim == 2:
    N_CHANNELS = 1
elif image.ndim == 3:
    N_CHANNELS = image.shape[-1]

# Create Dataset from folder

In [ ]:
# Returns the values of the function for images_set, labels_set on train-set
x_train, y_train = create_dataset(DATA_TRAIN_IMAGES, IMAGE_SHAPE = IMAGE_SHAPE)

target_dict_train = {k: v for v, k in enumerate(np.unique(y_train))}

y_train = [target_dict_train[y_train[i]] for i in range(len(y_train))]

In [ ]:
# Returns the values of the function for images_set, labels_set on test-set
x_test, y_test = create_dataset(DATA_TEST_IMAGES, IMAGE_SHAPE = IMAGE_SHAPE)

target_dict_test = {k: v for v, k in enumerate(np.unique(y_test))}

y_test = [target_dict_test[y_test[i]] for i in range(len(y_test))]

In [ ]:
# so we produce 3D matrix. But we require a 4D matrix to use tf.nn.conv2d for the convolutional layer. We increase the dimensions using tf.expand_dims.
# Só mudar explicação mudando para função do keras e expand_dims do numpy

# A 3D matrix was produced
# However, to use tf.nn.conv2d for the convolutional layer, a 4D matrix is needed,
# So tf.expand_dims is used to increase the dimension in the matrix
x_train = np.expand_dims(x_train, axis = 0)

x_test = np.expand_dims(x_test, axis = 0)

In [ ]:
# Before we jump into convolution, It’s necessary to know what will be the size of our filter
# matrix(which will slide over input data in the covolutional layer), it has to be a 4D tensor.
# num_filters and filter size are the hyperparameters you need to declare as per the requirements.
# The number of filters per filter size is num_filters . And often we use more than one filter
# size for convolution. For image data we can define it like:

#filter_shape = [filter_size num_input_channels 1 num_filters]

# Now we have to transpose this Tensor
# Só mudar explicação mudando para função do keras e expand_dims do numpy

# Before performing the convolution operation, it is necessary to know how
# the shape of the kernel Tensor must be:
# `filter_shape` = [filter_height, filter_width, in_channels, out_channels]

# So far, the shape of `feature_weights` is:
# `feature_weights` = [out_channels, filter_height, filter_width, in_channels]
# print(x_train.shape)

x_train = np.transpose(x_train, (1, 2, 3, 0))

x_test = np.transpose(x_test, (1, 2, 3, 0))

# One-Hot Encoded

In [ ]:
# One-Hot Encode
y_train = to_categorical(y_train)

y_test = to_categorical(y_test)

# Use a Manual Verification Dataset
## Split into sets for training and validation
## The validation-set will be used to validate the performance of the model (Cross-Validation)

In [ ]:
# Common reasons used:
# train 60%, val e test 20%
# train 70%, val e test 15%
# train 80%, val e test 10%
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train,
                                                  test_size = 0.1,
                                                  random_state = 41)

In [ ]:
# TODO: Criar uma função e jogar no módulo LOGFILE

print(f'[INFO] {(DATASET)} Dataset information:\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of Classes: {N_CLASSES}\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of Channels: {N_CHANNELS}\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of images in training-set: {len(x_train)}\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of images in validation-set: {len(x_val)}\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of images in test-set: {len(x_test)}\n', file = global_variables.LOGFILE)

print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)

# Save all sets as Numpy for easy future use

In [ ]:
# Training-set
np.save('/'.join([global_variables.OUTPUT_PATH, 'x_train']), x_train)
np.save('/'.join([global_variables.OUTPUT_PATH, 'y_train']), y_train)

In [ ]:
# Evaluation-set
np.save('/'.join([global_variables.OUTPUT_PATH, 'x_val']), x_val)
np.save('/'.join([global_variables.OUTPUT_PATH, 'y_val']), y_val)

In [ ]:
# Test-set
np.save('/'.join([global_variables.OUTPUT_PATH, 'x_test']), x_test)
np.save('/'.join([global_variables.OUTPUT_PATH, 'y_test']), y_test)

# Initiate detector and descriptor selected

In [ ]:
if USE_DESC_BLOCK == 'True':
    detector_initialized = find_detector_or_descriptor(DETECTOR)

    descriptor_initialized = find_detector_or_descriptor(DESCRIPTOR)

# Print detector and descriptor selected

In [ ]:
if USE_DESC_BLOCK == 'True':
    print_detector_initialized_info(detector = detector_initialized)

    print_descriptor_initialized_info(descriptor = descriptor_initialized)

# Model and Checkpoints data

In [ ]:
# Store and update epochs state
INIT_EPOCH_TRAIN = 0

# Output path for model and checkpoints
OUTPUT_MODEL_N_CHECK = "/".join([config['dataset'], config['architecture'], config['descriptor']])

# Where models are stored
MODEL_PATH = "/".join([BASE_PATH, 'Models', OUTPUT_MODEL_N_CHECK])

# Stores model data and checkpoints
CHECKPOINT_PATH = "/".join([BASE_PATH, 'Pretrained', OUTPUT_MODEL_N_CHECK])

try:
    os.makedirs(name = MODEL_PATH)
    print('\n[INFO]: `MODEL_PATH` does not exist... creating...')
except FileExistsError:
    # PRE_TRAINED_PATH already exists
    pass

try:
    os.makedirs(name = CHECKPOINT_PATH)
    print('\n[INFO]: `CHECKPOINT_PATH` does not exist... creating...')
except FileExistsError:
    # OUTPUT_PATH already exists
    pass

# Default name to save checkpoints
CHECKPOINT_FILENAME = os.path.join(CHECKPOINT_PATH, 'model_weights_improvement_{epoch:02d}_{val_acc:.2f}.h5')

# Used to check if checkpoints exists
CHECK_CHECKPOINT_FILENAME = '/'.join([CHECKPOINT_PATH, 'model_weights_improvement_**_*.**.h5'])

# TODO: Jogar essa função em algum lugar
# Help-function to get the initial epoch number from the checkpoint
def get_init_epoch_function(checkpoint_path):
    filename = os.path.basename(checkpoint_path)
    filename = os.path.splitext(filename)[0]
    
    init_epoch = filename.split("_")[3]
    
    return int(init_epoch)

# Check if checkpoints exists

In [ ]:
# if true, get the last one
if len(glob.glob(CHECK_CHECKPOINT_FILENAME)) != 0:
        # List of files
        list_of_files = glob.glob(CHECK_CHECKPOINT_FILENAME)

        # Get the last one checkpoint to load and continue from
        training_checkpoint = max(list_of_files, key = os.path.getmtime)

        # Get the epoch number to continue from
        INIT_EPOCH_TRAIN = get_init_epoch_function(checkpoint_path = training_checkpoint)

        print(f'[INFO] Training checkpoint found for epoch {INIT_EPOCH_TRAIN}. Will continue from that epoch\n', file = global_variables.LOGFILE)
        
        print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)

        load_from_checkpoint_train = True
else:
        print('[INFO] Training checkpoint not found. Will start from epoch 1\n', file = global_variables.LOGFILE)
        
        print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)
        
        load_from_checkpoint_train = False

# Local Feature Detection (LFD) Layer

In [ ]:
if USE_DESC_BLOCK == 'True':
    local_feature_detection(path_dir = DATA_TRAIN_IMAGES,
                            descriptor = DESCRIPTOR,
                            detector_initialized = detector_initialized,
                            descriptor_initialized = descriptor_initialized)
    
    print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)

# Local Descriptor Convolution (LDC) Layer

In [ ]:
with Timer() as timer:
    # Restore checkpoint
    if load_from_checkpoint_train:
        print(f'[INFO] Building Model:\n', file = global_variables.LOGFILE)

        json_file = '/'.join([MODEL_PATH, config['architecture'] + '.json'])

        json_file = open(json_file, 'r')

        loaded_model_json = json_file.read()

        json_file.close()

        if USE_DESC_BLOCK == 'True':
            with CustomObjectScope({'DescConv2D': DescConv2D}):
                    # Load Model
                    model = model_from_json(loaded_model_json)
                    
                    # Load Weights
                    model.load_weights(training_checkpoint)

        else:
            model = load_model(training_checkpoint) # TODO Talves mudar aqui também... testar com ResNet
    else:
      print(f'[INFO] Building Model:\n', file = global_variables.LOGFILE)

      model = ResNet(n_res = ARCHITECTURE, input_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS), classes = N_CLASSES)

print(f'[INFO] Time: {timer}\n', file = global_variables.LOGFILE)

model.summary(print_fn = lambda x: global_variables.LOGFILE.write(f'{x}\n'))

#print(f'\n', file = global_variables.LOGFILE)

In [ ]:
# Optimizer
# Printar esses dados também
opt = Adam(lr = 0.001,
           beta_1 = 0.9,
           beta_2 = 0.999,
           epsilon = 1e-8)

# if N_CLASSES == 2:
#    loss = 'sparse_categorical_crossentropy'
# else:
loss = 'categorical_crossentropy'

def RMSE(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1))

options = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)
metadata = tf.compat.v1.RunMetadata()

model.compile(loss = loss,
              optimizer = opt,
              #metrics = ['accuracy'],
              metrics = [RMSE, 'accuracy', 'top_k_categorical_accuracy'],
              options = options,
              run_metadata = metadata)

# Save Model Architecture Visualization

In [ ]:
save_architecture_visualization(model = model)

# Save Model

In [ ]:
# if USE_DESC_BLOCK:
h5_file = '/'.join([MODEL_PATH, ARCHITECTURE + '.h5'])
json_file = '/'.join([MODEL_PATH, ARCHITECTURE + '.json'])
# else:
#    h5_file = '/'.join([MODEL_PATH, ARCHITECTURE + '.h5'])
#    json_file = '/'.join([MODEL_PATH, ARCHITECTURE + '.json'])
    
# Save Model Architecture to H5
model.save(h5_file)

# Save Model Architecture to JSON
model_json = model.to_json()

with open(json_file, "w") as output_file:
    output_file.write(model_json)

# Define callbacks

In [ ]:
lr_reduce = ReduceLROnPlateau(monitor = 'val_acc',
                              factor = 0.9,
                              patience = 1,
                              verbose = 1)

# Avoid overfitting 
early_stop = EarlyStopping(monitor = 'val_acc',
                           min_delta = 0.001,
                           patience = 15,
                           verbose = 1,
                           mode = 'max',
                           restore_best_weights = True)


checkpoint = ModelCheckpoint(filepath = CHECKPOINT_FILENAME,
                             monitor = 'val_acc',
                             verbose = 1,
                             save_best_only = True,
                             mode = 'max',
                             period = 1)


CSV_FILENAME = '/'.join([global_variables.OUTPUT_PATH, 'epoch_logfile.csv'])

csv_log = CSVLogger(filename = CSV_FILENAME,
                    separator = ',',
                    append = False)

# schedule = StepDecay(initAlpha = 0.01,
#                      factor = 0.25, 
#                      dropEvery = 100)

schedule = CyclicLR(base_lr = 1e-7,
                    max_lr = 0.001,
                    step_size = 200)

callbacks = []

if not load_from_checkpoint_train:
    callbacks = [lr_reduce, early_stop, checkpoint, csv_log, schedule]
    #callbacks = [lr_reduce, checkpoint]

# Set hyperparameters from training

In [ ]:
N_ITERATIONS = int(config['n_iterations'])

BATCH_SIZE = int(config['batch_size'])

N_BATCHS = int(len(x_train) / BATCH_SIZE)

N_EPOCHS = int(N_ITERATIONS / N_BATCHS) #if N_BATCHS != 0 else 0

In [ ]:
# TODO: Jogar em outra função, a do LOGFILE?
print(f'\n[INFO] Hyperparameters from training:\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of iterations: {N_ITERATIONS}\n', file = global_variables.LOGFILE)

print(f'[INFO] Batch size: {BATCH_SIZE}\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of Batchs: {N_BATCHS}\n', file = global_variables.LOGFILE)

print(f'[INFO] Number of Epochs: {N_EPOCHS}\n', file = global_variables.LOGFILE)

print(f'{global_variables.LINE}\n', file = global_variables.LOGFILE)

# Data Augmentation
## In-place/on-the-fly data augmentation (most common)

In [ ]:
datagen = ImageDataGenerator(rotation_range = 20,
                              shear_range = 0.15,
                              zoom_range = 0.1,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              horizontal_flip = True,
                              fill_mode = 'nearest')
                              
print(len(datagen.flow(x_train, y_train)))

# Saving Augmented Images

In [ ]:
SAVE_AUG_FILE = '/'.join([global_variables.OUTPUT_PATH, 'Augmented Samples'])

# Create SAVE_AUG_FILE path if not exists
try:
    os.makedirs(name = SAVE_AUG_FILE)
    print('\n[INFO]: `SAVE_AUG_FILE` does not exist... creating...')
except FileExistsError:
    # SAVE_AUG_FILE already exists
    pass

for x_batch, y_batch in zip(datagen.flow(x_train,
                                         y_train,
                                         batch_size = 9,
                                         save_to_dir = SAVE_AUG_FILE,
                                         save_prefix = 'aug', 
                                         save_format = 'jpg'),
                                         range(100)):
    pass

# Training Model

In [ ]:
# Usarmos fit_generator() ao invés de fit() porque os dados de treinamento vieram de um gerador
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size = BATCH_SIZE),
                              epochs = N_EPOCHS,
                              verbose = 1,
                              validation_data = (x_val, y_val),
                              validation_steps = len(x_val) // BATCH_SIZE, 
                              steps_per_epoch = len(x_train) // BATCH_SIZE,
                              callbacks = callbacks)

In [ ]:
# history = training(model = model,
#                    X = x_train,
#                    Y = y_train,
#                    batch_size = BATCH_SIZE,
#                    n_epochs = N_EPOCHS,
#                    initial_epoch = INIT_EPOCH_TRAIN,
#                    X_val = x_val,
#                    Y_val = y_val,
#                    callbacks = callbacks)

# Generating improvement graph at each step of training

In [ ]:
# en-US default

# Use Matplotlib inline
%matplotlib inline

save_model_accuracy_history(model_history = history)

In [ ]:
# pt-BR

# Use Matplotlib inline
%matplotlib inline

save_model_accuracy_history(model_history = history, language = 'pt-BR')

In [ ]:
# en-US default

# Use Matplotlib inline
%matplotlib inline

save_model_loss_history(model_history = history)

In [ ]:
# pt-BR

# Use Matplotlib inline
%matplotlib inline

save_model_loss_history(model_history = history,
                        language = 'pt-BR')

# Evaluating the model on the test-set

In [ ]:
evaluate(model = model,
         X = x_test,
         Y = y_test,
         batch_size = BATCH_SIZE)

# Generating output predictions for the input data

In [ ]:
true_y, pred_y = predict(model = model,
                         X = x_test,
                         Y = y_test,
                         batch_size = BATCH_SIZE,
                         classes = CLASSES)

# Generating Confusion Matrix

In [ ]:
if DATASET != 'CIFAR-100' or DATASET != 'ILSVRC-2017':
    save_confusion_matrix(y_true = true_y,
                          y_pred = pred_y,
                          dataset = DATASET,
                          classes = CLASSES)

In [ ]:
if DATASET != 'CIFAR-100' or DATASET != 'ILSVRC-2017':
    save_confusion_matrix(y_true = true_y,
                          y_pred = pred_y,
                          dataset = DATASET,
                          classes = CLASSES,
                          language = 'pt-BR')

# Delete Dataset Folder

In [ ]:
if not use_colab:
    from shutil import rmtree
    rmtree(DATASET, ignore_errors = True)
else:
    pass

# Close logfile

In [ ]:
close_logfile()